# 🫁 Pneumothorax AI — Global Pre-training
**TÜBİTAK 2209-A | Ahmet Demir | Dokuz Eylül Üniversitesi**

**Veri stratejisi — Google Drive:**
- NIH verileri Drive'a bir kez indirilir, sonraki oturumlarda tekrar indirilmez
- Tüm checkpoint'ler Drive'a kaydedilir
- Drive path: `MyDrive/tubitak_pneumothorax/`

**Tahmini süre:** T4 GPU ile ~12 saat (50 epoch, 112k görüntü)

## 0. GPU Kontrolü

In [ ]:
import torch
print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK — Runtime > GPU seç!')
print('CUDA:', torch.version.cuda)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

## 1. Google Drive Bağla

## 2. Projeyi Klonla

## 1. Projeyi Klonla

## 3. Bağımlılıkları Yükle

## 2. Bağımlılıkları Yükle

## 4. Kaggle API Token Ayarla

## 3. Kaggle API Token Ayarla

## 5. NIH ChestX-ray14 — Drive'dan Yükle veya İndir

In [ ]:
import subprocess

NIH_LOCAL = 'data/raw/global/nih'
os.makedirs(NIH_LOCAL, exist_ok=True)

# ── 1) Drive'da veri var mı? ──────────────────────────────────────────────
drive_count = int(subprocess.check_output(
    f'find {DRIVE_NIH} -name "*.png" 2>/dev/null | wc -l', shell=True
).decode().strip())

if drive_count > 100000:
    print(f'✓ Drive\'da {drive_count:,} NIH görüntüsü bulundu — symlink oluşturuluyor...')
    # Colab'da hızlı erişim için sembolik bağlantı
    if not os.path.islink(NIH_LOCAL):
        os.rmdir(NIH_LOCAL)
        os.symlink(DRIVE_NIH, NIH_LOCAL)
    print(f'✓ {NIH_LOCAL} → {DRIVE_NIH}')

# ── 2) Drive'da yok → Kaggle'dan Drive'a indir ───────────────────────────
else:
    print(f'Drive\'da NIH bulunamadı — Kaggle\'dan Drive\'a indiriliyor (~42 GB)...')
    print('⚠️  Bu işlem ~30-60 dakika sürebilir, bir kez yapılır.')
    !kaggle datasets download \
        -d nih-chest-xrays/data \
        -p {DRIVE_NIH} \
        --unzip
    # Symlink
    if not os.path.islink(NIH_LOCAL):
        os.rmdir(NIH_LOCAL)
        os.symlink(DRIVE_NIH, NIH_LOCAL)
    downloaded = int(subprocess.check_output(
        f'find {DRIVE_NIH} -name "*.png" | wc -l', shell=True
    ).decode().strip())
    print(f'✓ İndirme tamamlandı: {downloaded:,} görüntü — Drive\'a kaydedildi')

## 6. Manifest Oluştur

## 5. Manifest Oluştur

## 7. W&B Giriş

## 6. W&B Giriş

## 8. Pre-training Başlat

**T4 GPU'da tahmini süre:**
- 1 epoch (112k görüntü, batch=32): ~14 dakika
- 50 epoch: ~12 saat

> ⚠️ Colab oturumu ~12 saatte kapanabilir. Checkpoint her fold'dan sonra kaydedilir.

In [ ]:
!python scripts/train_global.py \
    --sources NIH \
    --encoder efficientnet-b0 \
    --img_size 512 \
    --epochs 50 \
    --batch_size 32 \
    --num_folds 5 \
    --lr 1e-4 \
    --checkpoint_dir {DRIVE_CKPT}

## 8. Pre-training Başlat

**T4 GPU'da tahmini süre:**
- 1 epoch (112k görüntü, batch=32): ~14 dakika
- 50 epoch: ~12 saat

> ⚠️ Colab oturumu ~12 saatte kapanabilir. Checkpoint her fold'dan sonra kaydedilir.

## 9. Checkpoint'i Drive'a Kopyala

## 9. Checkpoint'i Drive'a Kopyala

In [ ]:
import shutil, glob

# Base model
base_model = 'checkpoints/global_base_model.pth'
if os.path.exists(base_model):
    shutil.copy(base_model, f'{DRIVE_CKPT}/global_base_model.pth')
    shutil.copy('checkpoints/global_base_model_meta.json',
                f'{DRIVE_CKPT}/global_base_model_meta.json')
    print(f'✓ Base model Drive\'a kopyalandı')

# Tüm fold checkpoint'leri
for ckpt in glob.glob('checkpoints/global_folds/*.pth'):
    shutil.copy(ckpt, DRIVE_CKPT)
    print(f'  → {os.path.basename(ckpt)}')

# Results CSV
results_csv = 'results/global_kfold_results.csv'
if os.path.exists(results_csv):
    shutil.copy(results_csv, DRIVE_CKPT)
    print(f'✓ Results CSV kopyalandı')

print(f'\n✓ Tüm dosyalar Drive\'a kaydedildi: {DRIVE_CKPT}')

In [ ]:
import pandas as pd
import json

# K-Fold sonuçları
try:
    df = pd.read_csv('results/global_kfold_results.csv')
    print('=== K-FOLD SONUÇLARI ===')
    print(df.to_string(index=False))
    print(f'\nOrtalama Dice : {df["best_dice"].mean():.4f} ± {df["best_dice"].std():.4f}')
    print(f'Ortalama AUC  : {df["best_auc"].mean():.4f} ± {df["best_auc"].std():.4f}')
except FileNotFoundError:
    print('Results CSV bulunamadı — eğitim henüz bitmemiş olabilir')

# Base model meta
try:
    meta = json.load(open('checkpoints/global_base_model_meta.json'))
    print('\n=== BASE MODEL ===')
    for k, v in meta.items():
        print(f'  {k}: {v}')
except FileNotFoundError:
    pass

---
## ✅ Sonraki Adım — Fine-tuning

DEU DICOM + NRRD verileri geldiğinde:

```bash
# Checkpoint'i Drive'dan al
cp /content/drive/MyDrive/tubitak_pneumothorax/checkpoints/global_base_model.pth checkpoints/

# DEU verilerini koy
# data/local/dicom/*.dcm
# data/local/nrrd/*.nrrd

# Fine-tune et (~1-2 saat)
python scripts/fine_tune_local.py --freeze_encoder --epochs 20
```